# 리눅스

In [1]:
# 기본 작업 경로 설정
import os
notebook_path = os.path.abspath("project_3_git/code/3_lm_t5.ipynb")
notebook_dir = os.path.dirname(notebook_path)
os.chdir(notebook_dir)

# 현재 작업 디렉토리 출력
print("Current working directory: ", os.getcwd())

Current working directory:  /mnt/e/py_data/project_3_git/code


In [ ]:
import cv2
from ultralytics import YOLO

test_image = cv2.imread('../data/image_data/test/image/happy/1k53eec98dfe2ec7d6f62c7d7c0ec5dc6d8ee4c6d6e8d60ed0076e290fcceu1ww.jpg')
# 훈련된 모델 불러오기

model_from_pret = YOLO('models/yolov8x-oiv7.pt')

model_from_pret(test_image)[0].show()

In [43]:
result = model_from_pret(test_image)[0]


0: 480x640 1 Car, 1 Clothing, 1 Human face, 2 Mans, 1 Tree, 1 Truck, 3 Wheels, 332.5ms
Speed: 5.6ms preprocess, 332.5ms inference, 2.7ms postprocess per image at shape (1, 3, 480, 640)


In [ ]:
import cv2
from ultralytics.models import YOLOv10

# 훈련된 모델 불러오기

model_transfered = YOLOv10('../yolov10/runs/detect/train/weights/last.pt')

model_transfered(test_image)[0].show()

In [7]:
result = model_transfered(test_image)[0]


0: 288x512 1 anger, 2 happys, 71.9ms
Speed: 2.1ms preprocess, 71.9ms inference, 5.4ms postprocess per image at shape (1, 3, 288, 512)


In [8]:
detections = result.boxes
emotion_mapping = {
    0 : '분노',
    1 : '슬픔',
    2 : '공포',
    3 : '기쁨',
}

In [9]:
text_intput = [emotion_mapping[int(i)] for i in detections.cls]

text_intput_text = ''
for i in text_intput:
    text_intput_text +=i + ','

text_intput_text = text_intput_text[:-1]

In [12]:
text_intput_text

'기쁨,기쁨,분노'

In [11]:
from transformers import T5TokenizerFast, T5ForConditionalGeneration
import torch

# 저장된 모델과 토크나이저 로드
model_save_path = '../model_follow_up/t5/model/'
tokenizer_save_path = '../model_follow_up/t5/model/tokenizer/'

model = T5ForConditionalGeneration.from_pretrained(model_save_path)
tokenizer = T5TokenizerFast.from_pretrained(tokenizer_save_path)

# 테스트 입력
test_input = text_intput_text
# 입력 토큰화
input_ids = tokenizer.encode(test_input, return_tensors='pt')

# 모델 예측
with torch.no_grad():
    outputs = model.generate(input_ids, max_length=50, num_beams=4, early_stopping=True)

# 예측 결과 디코딩
predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"테스트 입력: {test_input}")
print(f"모델의 예측: {predicted_text}")


테스트 입력: 기쁨,기쁨,분노
모델의 예측: 기쁨의 표정을 보니 멋진 자연을 만끽하고 계시네요!
